
# POC 激活验证

### 1. 同构激活图

In [1]:
import torch 
params = torch.load('./shadow_model_ckpt/mnist/models5/shadow_benign_0.model')
from model_lib.mnist_cnn_model import Model5 as Model
m = Model()
m.load_state_dict(params)
print(m)

/home/dorian/.conda/envs/mntd/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Model5(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(16, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=392, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (output): Linear(in_features=128, out_features=10, bias=True)
)


In [2]:
import dgl
import torch
from model_lib.hetero_struct_backdoor_dataset import HeteroStrucBackdoorDataset
from utils_gnn import all_u_to_v


def cnn2graph_activation(model, model_info):
    # convert cnn model to a dgl graph
    # model: model weight data
    # model_info: model struct info
    
    layers = []
    all_edges = []


    node_layer_idx = []
    all_node_feats = []
    pre_node_size = []
    node_params = []
    all_bias_feats = []
    pre_bias_size = []
    node_layer_num = []

    pooling = []
    
    input_layer = True
    concat_layer = True
    idx = 0
    cnt = 0
    with torch.no_grad():
        for i in range(len(model_info)):
            cur_layer_info = model_info[i]
            pooling_info = cur_layer_info['maxpool']
            print(pooling_info)
            cur_layer_node = []
            if not input_layer and idx == 0:
                idx += 1
            if input_layer:
                input_layer = False
            if 'conv' in cur_layer_info['name']:
                # construct cur layer nodes
                for weight, bias in zip(model.get_submodule(cur_layer_info['name']).weight, 
                                        model.get_submodule(cur_layer_info['name']).bias):
                    # print("cur layer info:", model.get_submodule(cur_layer_info['name']))
                    cur_layer_node.append(cnt)
                    cnt += 1
                    if pooling_info:
                        pooling.append([pooling_info['kernel_size'], pooling_info['stride'],
                        pooling_info['padding'], pooling_info['dilation'], int(pooling_info['ceil_mode'])])
                    else:
                        pooling.append([0, 0, 0, 0, 0])
                    node_layer_idx.append(idx)
                    node_layer_num.append(0)
                    # params 
                    conv = model.get_submodule(cur_layer_info['name'])
                    params = [conv.kernel_size, conv.stride, conv.padding]
                    node_params.append(params)
                    # print(params)
                    # featue resize?
                    w = weight[0]
                    r, c = w.size()
                    pre_node_size.append([r, c])
                    all_node_feats.append(padding(w, 512, 512))
                    b = bias.expand(1,1)
                    r, c = b.size()
                    # print("size:", b.size())
                    pre_bias_size.append([r, c])
                    all_bias_feats.append(padding(b, 1, 512))
                    # print("conv weight:", w.shape)
                    # bias
                    # print("conv bias:", bias)
            else:
                # construct dense layer node
                pooling.append([0, 0, 0, 0, 0])
                node_layer_num.append(1)
                cur_layer_node.append(cnt)
                cnt += 1
                if concat_layer:
                    idx += 1
                    concat_layer = False
                    node_layer_idx.append(idx)
                    idx += 1
                else:
                    node_layer_idx.append(idx)
                params = [(0, 0), (0, 0), (0, 0)]
                node_params.append(params)
                # feature resize?
                weight = model.get_submodule(cur_layer_info['name']).weight.t()
                bias = model.get_submodule(cur_layer_info['name']).bias
                w = weight
                r, c = w.size()
                pre_node_size.append([r, c])
                all_node_feats.append(padding(w, 512, 512))
                b = bias.expand(1,-1)
                r, c = b.size()
                # print("size:", b.size())
                pre_bias_size.append([r, c])
                all_bias_feats.append(padding(b, 1, 512))
            layers.append(cur_layer_node)
            
    # get all edges
    for idx in range(len(layers)):
        if idx < len(layers) - 1:
            edges = all_u_to_v(layers[idx], layers[idx+1])
            all_edges += edges
    
    all_edges = torch.tensor(all_edges).t()
    u, v = all_edges[0], all_edges[1]
    g = dgl.graph((u,v)).to('cuda')
    g.ndata['x'] = torch.stack(all_node_feats)
    # tag for message transmission process
    g.ndata['tag'] = torch.tensor(node_layer_idx).to('cuda')
    # layer for layer type, 0 for conv, 1 for full connect
    g.ndata['layer'] = torch.tensor(node_layer_num).to('cuda')
    # acutal node size(kernel size or fc node size)
    g.ndata['node_size'] = torch.tensor(pre_node_size).to('cuda')
    # params for conv kernel params 
    g.ndata['params'] = torch.tensor(node_params).to('cuda')
    # bias weight
    g.ndata['bias'] = torch.stack(all_bias_feats)
    # bias size
    g.ndata['bias_size'] = torch.tensor(pre_bias_size).to('cuda')
    # pooling params
    g.ndata['pooling'] = torch.tensor(pooling).to('cuda')
    
    return g



In [3]:
from dgl.dataloading import GraphDataLoader
from dgl.heterograph import DGLHeteroGraph
from model_lib.hetero_struct_backdoor_dataset import HeteroStrucBackdoorDataset
import torch
def test_hetero_activation():
    dataset = HeteroStrucBackdoorDataset(nums=10,
        raw_dir='/home/dorian/repos/Meta-Nerual-Trojan-Detection/shadow_model_ckpt/mnist/models', activation=True)
    dataloader = GraphDataLoader(dataset, batch_size=1, pin_memory=torch.cuda.is_available())
    for batch, (batched_graph, labels) in enumerate(dataloader):
        assert batch == 0
        assert isinstance(batched_graph, DGLHeteroGraph)
        # assert batched_graph.ndata['idx'].any()
        # assert batched_graph.ndata['node_size']
        # assert batched_graph.ndata['params']
        # assert batched_graph.ndata['bias']
        # assert batched_graph.ndata['bias_size']
        print(batched_graph)
        break

test_hetero_activation() 


Graph(num_nodes=162, num_edges=5153,
      ndata_schemes={'x': Scheme(shape=(512, 512), dtype=torch.float32), 'idx': Scheme(shape=(), dtype=torch.int64), 'node_size': Scheme(shape=(2,), dtype=torch.int64), 'params': Scheme(shape=(3, 2), dtype=torch.int64), 'bias': Scheme(shape=(1, 512), dtype=torch.float32), 'bias_size': Scheme(shape=(2,), dtype=torch.int64)}
      edata_schemes={})


### 2. 设计传播函数模拟神经网络传参
- 图片分类网络结果  
    - [x] 读取数据
    - [x] 调用分类网络获得分类结果
    - [x] 获取网络中间结果  
- 设计传播函数  
    - [ ] conv 层
    - [ ] 全连接层
    - [ ] 输出层获得分类结果

In [2]:
# from model_lib import mnist_cnn_model as father_model
from utils_basic import load_spec_model
from utils_gnn import padding
x = '/home/dorian/repos/Meta-Nerual-Trojan-Detection/shadow_model_ckpt/mnist/models5/shadow_jumbo_9.model'
# load model 
# Model = load_spec_model(father_model, '5')
from model_lib.mnist_cnn_model import Model6 as Model
model = Model(gpu=True)
params = torch.load(x)
model.load_state_dict(params)

# load model detail 
# model_detail = {}
# model_detail_path = "./intermediate_data/model_detail.json"
# import json
# with open(model_detail_path, 'r') as f:
#     model_detail = json.load(f)
# print(model_detail)
# g = cnn2graph_activation(model, model_detail['mnist']['5'])
# dgl.save_graphs('./intermediate_data/grapj_test.bin', g)

<All keys matched successfully>

In [6]:
import torchvision
import torchvision.transforms as transforms
# from utils_gnn import SGNACT
GPU = True
if GPU:
        torch.cuda.manual_seed_all(0)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        
transform = transforms.Compose([
            transforms.ToTensor(),
        ])
BATCH_SIZE = 1
# MNIST image dataset 
trainset = torchvision.datasets.MNIST(root='./raw_data/', train=True, download=True, transform=transform)
dataloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE)

from model_lib.hetero_struct_backdoor_dataset import HeteroStrucBackdoorDataset
from dgl.dataloading import GraphDataLoader
# Shadow model dataset
dataset = HeteroStrucBackdoorDataset(nums=10,
    raw_dir='/home/dorian/repos/Meta-Nerual-Trojan-Detection/shadow_model_ckpt/mnist/models', activation=True)
model_dataloader = GraphDataLoader(dataset, batch_size=1, pin_memory=torch.cuda.is_available())



In [10]:
import dgl
from dgl.data import DGLDataset
import os
from tqdm import tqdm
from dgl import save_graphs, load_graphs
import torch
import json
from torch import nn
import numpy as np
# from model_lib.mnist_cnn_model import Model
from random import randint
from utils_basic import load_spec_model
from sklearn.metrics import roc_auc_score
from torchinfo import summary
import torch.nn.functional as F
from dgl.nn.pytorch.conv import GINConv
from dgl.nn.pytorch.glob import SumPooling, AvgPooling, MaxPooling, SortPooling
from utils_gnn import MLP
from utils_gnn import unpadding, padding


def conv(data, weight, bias, kernel_size, stride, padding):
    row, col = weight.size()
    # get actual conv kernel weight and bias
    w = unpadding(weight, kernel_size, kernel_size)
    w = w.unsqueeze(0).unsqueeze(0)
    b = unpadding(bias, 1, 1)[0]
    # get conv operator 
    operator = torch.nn.Conv2d(1, 1, kernel_size=kernel_size, stride=stride,
                    padding=padding)
    # set conv operator weight and bias
    operator.weight.data = w
    operator.bias.data = b
    # conduct conv operation
    x = operator(data)
    return padding(x[0][0], )

def maxpool(kernel_size, stride, padding):
    pass

def message_func(edges):
    print("this is message function.")
    print("message fucntion ends")
    return {'pre': edges.src['x']}

def reduce_func(nodes):
    print("this is reduce function")
    input_mask = nodes.data['tag'] == 0
    conv_mask = nodes.data['tag'] == 1
    concat_mask = nodes.data['tag'] == 2
    fc_mask = nodes.data['tag'] == 3

    # input_feat = nodes.data['x'][input_mask]
    # print(input_feat)
    print("reduce function ends")
    return {'pre': nodes.mailbox['pre']}


def cnn_cal(graph):
    graph.update_all(message_func=message_func, reduce_func=reduce_func)
    ft = graph.ndata['ft'][0]
    return ft

model.eval()
for batch, (batched_graph, labels) in enumerate(model_dataloader):
    print(batched_graph)
    feat = batched_graph.ndata.pop('x')
    for i, (x_in, y_in) in enumerate(dataloader):
        # print(x_in.size())
        # get orginal cnn calculation resutls
        pred, params = model(x_in)
        print(torch.argmax(pred))
        # get calculation results on graph node 
        res = cnn_cal(batched_graph)
        # compare
        break
    break

Graph(num_nodes=162, num_edges=5153,
      ndata_schemes={'x': Scheme(shape=(512, 512), dtype=torch.float32), 'idx': Scheme(shape=(), dtype=torch.int64), 'node_size': Scheme(shape=(2,), dtype=torch.int64), 'params': Scheme(shape=(3, 2), dtype=torch.int64), 'bias': Scheme(shape=(1, 512), dtype=torch.float32), 'bias_size': Scheme(shape=(2,), dtype=torch.int64)}
      edata_schemes={})
tensor(3, device='cuda:0')


KeyError: 'x'

In [36]:
x_in = x_in.to('cuda')
x = conv(x_in, g.ndata['x'][0], g.ndata['bias'][0], 5, 1, 2)
print(x.size())

254 253 254 253
256 255 0 -1


TypeError: 'int' object is not callable

In [25]:
bias = g.ndata['bias'][0]
b = unpadding(bias, 1, 1)

256 255 0 -1


In [28]:
b[0][0]

tensor(0.0161, device='cuda:0')

In [31]:
model.conv1.bias.data.size()

torch.Size([32])